# Finance 5350 

__Computational Financial Modeling__

## Introduction to the Black-Scholes-Merton Model

The BSM formula is given by the following (for a call option):

$$
C(S, K, \sigma,r, T, \delta) = S e^{-\delta T} N(d_{1}) - K e^{-r T} N(d_{2})
$$

<br>

and by (for a put option):

$$
P(S, K, \sigma,r, T, \delta) = K e^{-rT}N(-d_{2}) - S e^{-\delta T} N(-d_{1})
$$

<br>

where

$$
\begin{align}
d_{1} &= \frac{\ln{(S/K)} + (r - \delta + \frac{1}{2} \sigma^{2}) T}{\sigma \sqrt{T}} \\
d_{2} &= d_{1} - \sigma \sqrt{T}
\end{align}
$$

<br>

Let's see the call option formula as a pricer function:

In [1]:
import numpy as np
from scipy.stats import norm

In [2]:
def blackScholesCall(spot, strike, vol, rate, expiry, div):
    d1 = (np.log(spot / strike) + (rate - div + 0.5 * vol * vol) * expiry) / (vol * np.sqrt(expiry))
    d2 = d1 - (vol * np.sqrt(expiry))
    callPrc = (spot * np.exp(-div * expiry) * norm.cdf(d1)) - (strike * np.exp(-rate * expiry) * norm.cdf(d2))
    
    return callPrc

In [3]:
## Basic Data (see McDonald Chp 12 page 377 2nd Edition)
spot = 41.0
strike = 40.0
vol = 0.30
rate = 0.08
expiry = 0.25
div = 0.0

callPrc = blackScholesCall(spot, strike, vol, rate, expiry, div)
callPrc

3.3990781872368956

In [5]:
## Call Payoff Function
def callPayoff(spot, strike):
    return np.maximum(spot - strike, 0.0)

## Put Payoff Function
def putPayoff(spot, strike):
    return np.maximum(strike - spot, 0.0)

In [8]:
from scipy.stats import binom

def binomialPricer(S, K, r, v, q, T, n, payoff, verbose = True):
    nodes = n  + 1
    h = T / n
    u = np.exp((r - q) * h + v * np.sqrt(h))
    d = np.exp((r - q) * h - v * np.sqrt(h))
    pstar = (np.exp((r - q) * h) - d) / (u - d)
    
    price = 0.0
    
    for i in range(nodes):
        prob = binom.pmf(i, n, pstar)
        spotT = S * (u ** i) * (d ** (n - i))
        po = payoff(spotT, K) 
        price += po * prob
        if verbose:
            print(f"({spotT:0.4f}, {po:0.4f}, {prob:0.4f})")
        
    price *= np.exp(-r * T)
    
    return price


In [10]:
N = [10, 25, 50, 75, 100, 125, 150, 175, 200, 500]

for i in range(len(N)):
    nsteps = N[i]
    callPrc = binomialPricer(spot, strike, rate, vol, div, expiry, nsteps, callPayoff, verbose=False)
    print(callPrc)

3.4544402138080796
3.408694981783251
3.3917972818360145
3.401464570319705
3.4046182898109367
3.3993570672462536
3.399604531287815
3.4022193238912726
3.3982871838738475
3.399919762643824
